<html>
<head>
<style>
h1 {text-align: center;}
p {text-align: center;}
div {text-align: center;}
</style>
</head>
<body>

<h1>Data analysis</h1>

<h3>Asbjørn Fyhn & Emil Kolko Beckett</h3>
<h3>Seminar: Topics in Sovereign Debt (Spring 2025)</h3>

</body>
</html>

## Notes for next time: 
- section 5
    - Remove observations after 2018
    - Predict spreads: Beregn som i en binomial model: $S_{up}$ (value 1) og $S_{down}$ (recovery rate)
    - Opstill figurer etc

- Ryd op i analysis of process raw: 
    - flyt spread beregning til prossess raw 
    - Regn YTD rigtigt, lav en YTD_capped, dummy for capped
    - Brug den nye RISK_REGION i stedet for coun_region_dict

In [ ]:
# automatic reload of modules
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np
from datetime import datetime
import statsmodels.api as sm; 
import LinearModel as lm, estimation, tools
from copy import deepcopy
import matplotlib.pyplot as plt
from scipy.stats import t, norm
import math
import warnings

# Function to determine the sign of a number
sign = lambda x: math.copysign(1, x) # returns 1 if x > 0 else -1 if x < 0 else 0

# Boolean options for saving figures and tables
save_figures = True

# create a list of hex colors for each region
cmap = tools.colorway

plt.rcParams['font.family'] = 'serif' #'Century Gothic'
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_to_data = r'/Users/asbjornfyhn/Desktop/Seminar - sovergin debt/code/data/'
path_to_output = r'/Users/asbjornfyhn/Desktop/Seminar - sovergin debt/code/output/'

# Load data 

In [ ]:
df_embi = pd.read_excel(path_to_data+'embi_regression_data.xlsx')
# Create YTD properly and constant 
ysd_var = 'years_since_in_default' # years_since_new_default, years_since_in_default # last one yield better results
df_embi['YEARS_SINCE_DEFAULT'] = df_embi[ysd_var].copy(deep=True) 
df_embi.loc[(df_embi[ysd_var]>=11) | (df_embi[ysd_var].isna()),'YEARS_SINCE_DEFAULT'] = 11.

df_embi['constant'] = 1
df_embi['spread'] = df_embi['spread'] *100
# Renaming to make it compatible with the rest of the code
df_embi = df_embi.rename(
    columns={
        'Gold_to_GDP':'gold_to_gdp', 
        'External Debt to GDP':'ex_debt_to_gdp', 
        'YEARS_SINCE_DEFAULT':'YTD',
        'PCT_TOT_Real':'PCT_TOT_Real_5yr',
        'VOL_TOT_Real':'VOL_TOT_Real_5yr',}
)

In [ ]:
df = pd.read_excel(path_to_data+'/prob_model_data.xlsx',index_col=[0,1])
df['constant'] = 1

bUnique = df.loc[(df.index.get_level_values(1)<='2000-01-01'),].index.get_level_values(0).unique()
# df = df.loc[df.index.get_level_values(0).isin(bUnique),]
df_embi['Date'] = pd.to_datetime(df_embi['Date'])
df = df.loc[df.index.get_level_values(1) < df_embi['Date'].min()]

print(f"Number of defaults {df['New_default'].sum():.0f}")
df.loc[df['New_default']==1,:]


In [ ]:
y_str = 'New_default'
x_str = ['constant','PCT_TOT_Real_5yr','VOL_TOT_Real_5yr','YTD','ex_debt_to_gdp','gold_to_gdp']
x_labels_3y = ['constant','PCT_TOT_Real_3yr','VOL_TOT_Real_3yr','YTD','ex_debt_to_gdp','gold_to_gdp']

pltDf = df.groupby(level=1)[['New_default','constant']].sum()
pltDf1 = df.dropna(subset=[y_str]+x_str,how='any').groupby(level=1)[['New_default','constant']].sum()
pltDf2 = df.dropna(subset=[y_str]+x_labels_3y,how='any').groupby(level=1)[['New_default','constant']].sum()

# Calculate non-defaults
non_defaults = pltDf['constant'] - pltDf['New_default']

# Create the stacked bar chart
fig, ax = plt.subplots(figsize=(10, 5))

# Stack defaults on top
ax.bar(pltDf.index.year, pltDf['New_default'], color='red', label='Defaults',alpha=0.4)
ax.scatter(pltDf1.index.year, pltDf1['New_default'], color='black', label='Defaults',alpha=0.8, marker='s')
ax.scatter(pltDf2.index.year, pltDf2['New_default'], color='white', label='Defaults',alpha=0.8, marker='x')
# Plot non-defaults as the base
# ax.bar(pltDf.index.year, non_defaults, bottom=pltDf['New_default'], color='blue', label='Non-Defaults', alpha=0.4)

# Customize
ax.set_ylabel('Number of Observations')
ax.set_xlabel('Year')
ax.set_title('Stacked Bar Chart of Defaults and Non-Defaults')
ax.legend()

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Adjust layout
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
class logitus:
    def __init__(self, df:pd.DataFrame, yLabel:str, xLabels:list, ):
        self.yLabel = yLabel
        self.xLabels = xLabels

        self.df = deepcopy(df.dropna(subset=[yLabel]+xLabels,how='any'))
        print(
            self.df.index.get_level_values(1).max()
        )
        self.x = deepcopy(self.df[xLabels].values)
        self.y = deepcopy(self.df[yLabel].values)

    def describe_y(self,verbose:bool=True):
        y = deepcopy(self.y)
        self.no_of_obs = y.shape[0]
        self.no_of_defaults = y[y==1].shape[0]
        self.no_of_non_defaults = y[y==0].shape[0]
        if verbose:
            print(f"Number of observations: {self.no_of_obs}")
            print(f"Number of defaults: {self.no_of_defaults}")
            print(f"Number of non-defaults: {self.no_of_non_defaults}")

    def remove_outliers(self, outliar_cut=3, indices=None):
        X = deepcopy(self.x)
        
        if indices is None:
            indices = [i for i in range(X.shape[1]) if self.xLabels[i] != 'constant']

        # Calculate mean and standard deviation along axis 0 (across rows for each column)
        means = np.mean(X[:, indices], axis=0)
        stds = np.std(X[:, indices], axis=0)

        # Create a boolean mask for rows where all values are within outliar_cut standard deviations
        mask = np.all(np.abs(X[:,indices] - means) <= (outliar_cut * stds), axis=1)

        # Apply the mask to filter the array
        X_filtered = X[mask]
        y_filtered = self.y[mask]

        self.x = deepcopy(X_filtered)
        self.y = deepcopy(y_filtered)

    def run_logit(self, display:bool=True):
        """ """
        # Get initial values for theta (using OLS)
        theta0 = tools.starting_values(self.y, self.x)
        
        # 
        logit_res = estimation.estimate(
            q = tools.q,
            theta0=theta0,
            y=self.y,
            x=self.x,
            cov_type='Sandwich',#'Sandwich',
            options={'disp':display}
        )

        regRes = pd.DataFrame(index=['beta','se','t-stat'], columns=self.xLabels, data=[logit_res[col] for col in ['theta','se','t']]).T
        degressFreedom = self.y.shape[0]-len(self.xLabels)-1
        regRes['p'] = 2*(t.cdf(-abs(regRes['t-stat']), degressFreedom))
        
        self.logit_res = logit_res
        self.regressio_results = regRes
        return regRes

    def ape(self):
        """ """
        apeRes = pd.DataFrame(index=self.xLabels, columns=['ape'])
        for i, label in enumerate(self.xLabels):
            if label == 'constant':
                continue
            ape = tools.average_partial_effect(x=self.x, 
                                               betas=self.logit_res['theta'],
                                               k=i)
            apeRes.loc[label,'ape'] = ape
            del ape
        
        return apeRes

## Run regression

In [ ]:
df = df.loc[df['RISK_REGION']!='Middle East']
outliar_cut = 2
small_indices = [1,2]
large_indices = [0,1,2,4,5]
large_indices = [4]
verbose = True
dispaly = False

In [ ]:
x_5yr_small = ['constant','PCT_TOT_Real_5yr','VOL_TOT_Real_5yr','YTD']
l_5yr_small = logitus(df, yLabel='New_default', xLabels=x_5yr_small)
l_5yr_small.remove_outliers(outliar_cut=outliar_cut, indices=small_indices)
l_5yr_small.describe_y(verbose=verbose)
_ = l_5yr_small.run_logit(display=False)
ape_5yr_small = l_5yr_small.ape()
print(ape_5yr_small)


# # Run bootstrap
# apes, ses = [],[]
# for i, label in enumerate(x_5yr_small):
#     if label == 'constant':
#         continue
#     res = tools.bootstrap_se_with_fit(l_5yr_small.x, l_5yr_small.y,k=i,n_boot=1000, seed=2)
#     apes.append(res[0])
#     ses.append(res[1])

# print(apes)
# print(ses)

In [ ]:
x_3yr_small = ['constant','PCT_TOT_Real_3yr','VOL_TOT_Real_3yr','YTD']
l_3yr_small = logitus(df, yLabel='New_default', xLabels=x_3yr_small)
l_3yr_small.remove_outliers(outliar_cut=outliar_cut, indices=small_indices)
l_3yr_small.describe_y(verbose=verbose)
_ = l_3yr_small.run_logit(display=False)
ape_3yr_small = l_3yr_small.ape()

# # Run bootstrap
# apes, ses = [],[]
# for i, label in enumerate(x_3yr_small):
#     if label == 'constant':
#         continue
#     res = tools.bootstrap_se_with_fit(l_3yr_small.x, l_3yr_small.y,k=i,n_boot=1000, seed=2)
#     apes.append(res[0])
#     ses.append(res[1])

# print(apes)
# print(ses)

In [ ]:
x_5yr_full = ['constant','PCT_TOT_Real_5yr','VOL_TOT_Real_5yr','YTD','ex_debt_to_gdp','gold_to_gdp']
l_5yr_full = logitus(df, yLabel='New_default', xLabels=x_5yr_full)
l_5yr_full.remove_outliers(outliar_cut=outliar_cut, indices=large_indices)
l_5yr_full.describe_y(verbose=verbose)
_ = l_5yr_full.run_logit(display=False)
ape_5yr = l_5yr_full.ape()

# # Run bootstrap
# apes, ses = [],[]
# for i, label in enumerate(x_5yr_full):
#     if label == 'constant':
#         continue
#     res = tools.bootstrap_se_with_fit(l_5yr_full.x, l_5yr_full.y,k=i,n_boot=1000, seed=2)
#     apes.append(res[0])
#     ses.append(res[1])

# print(apes)
# print(ses)

In [ ]:
x_3yr_full = ['constant','PCT_TOT_Real_3yr','VOL_TOT_Real_3yr','YTD','ex_debt_to_gdp','gold_to_gdp']
l_3yr_full = logitus(df, yLabel='New_default', xLabels=x_3yr_full)
# l_3yr_full.remove_outliers(outliar_cut=2, indices=[1,2,4,5])
l_3yr_full.remove_outliers(outliar_cut=outliar_cut, indices=large_indices)
l_3yr_full.describe_y(verbose=verbose)
_ = l_3yr_full.run_logit(display=False)
ape_3yr = l_3yr_full.ape() 


# # Run bootstrap
# apes, ses = [],[]
# for i, label in enumerate(x_3yr_full):
#     if label == 'constant':
#         continue
#     res = tools.bootstrap_se_with_fit(l_3yr_full.x, l_3yr_full.y,k=i,n_boot=1000, seed=2)
#     apes.append(res[0])
#     ses.append(res[1])

# print(apes)
# print(ses)

In [ ]:
dfRes = pd.concat(
    [
    l_3yr_full.regressio_results[['beta','se','p']].stack().to_frame().rename(columns={0:'3 Yr full'}).rename(index={'PCT_TOT_Real_3yr':'PCT_TOT','VOL_TOT_Real_3yr':'VOL_TOT'}),
    l_3yr_small.regressio_results[['beta','se','p']].stack().to_frame().rename(columns={0:'3 Yr small'}).rename(index={'PCT_TOT_Real_3yr':'PCT_TOT','VOL_TOT_Real_3yr':'VOL_TOT'}),
    l_5yr_full.regressio_results[['beta','se','p']].stack().to_frame().rename(columns={0:'5 Yr full'}).rename(index={'PCT_TOT_Real_5yr':'PCT_TOT','VOL_TOT_Real_5yr':'VOL_TOT'}),
    l_5yr_small.regressio_results[['beta','se','p']].stack().to_frame().rename(columns={0:'5 Yr small'}).rename(index={'PCT_TOT_Real_5yr':'PCT_TOT','VOL_TOT_Real_5yr':'VOL_TOT'}),
    ], axis=1)

# add number of observations to the table
dfRes.loc[('Total','Obs'), :] = [l_3yr_full.no_of_obs,
                                             l_3yr_small.no_of_obs,
                                             l_5yr_full.no_of_obs,
                                             l_5yr_small.no_of_obs]
# add number of defaults to the table
dfRes.loc[('Defaults','Obs'), :] = [l_3yr_full.no_of_defaults,
                                             l_3yr_small.no_of_defaults,
                                             l_5yr_full.no_of_defaults,
                                             l_5yr_small.no_of_defaults]
# add number of non-defaults to the table
dfRes.loc[('Non-Defaults','Obs'), :] = [l_3yr_full.no_of_non_defaults,
                                             l_3yr_small.no_of_non_defaults,
                                             l_5yr_full.no_of_non_defaults,
                                             l_5yr_small.no_of_non_defaults]

if save_figures:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        resLatexy = dfRes.to_latex()
        warnings.simplefilter("default")
    for i in range(len(resLatexy)):
        resLatexy = resLatexy.replace('NaN','-')
    resLatexy = resLatexy.replace('Obs','')

    with open(path_to_output+'logit_results.tex', 'w') as f:
        f.write(resLatexy)
    print('Saved results to logit_results.tex')

dfRes

In [ ]:
ape = pd.concat([
    ape_3yr.rename(index={'PCT_TOT_Real_3yr':'Chg. ToT','VOL_TOT_Real_3yr':'Vol. ToT', 'ex_debt_to_gdp':'Ext. Debt', 'gold_to_gdp':'Curr. res.'},columns={'ape':'3 Yr full'}),
    ape_3yr_small.rename(index={'PCT_TOT_Real_3yr':'Chg. ToT','VOL_TOT_Real_3yr':'Vol. ToT', 'ex_debt_to_gdp':'Ext. Debt', 'gold_to_gdp':'Curr. res.'},columns={'ape':'3 Yr small'}),
    ape_5yr.rename(index={'PCT_TOT_Real_5yr':'Chg. ToT','VOL_TOT_Real_5yr':'Vol. ToT', 'ex_debt_to_gdp':'Ext. Debt', 'gold_to_gdp':'Curr. res.'},columns={'ape':'5 Yr full'}), 
    ape_5yr_small.rename(index={'PCT_TOT_Real_5yr':'Chg. ToT','VOL_TOT_Real_5yr':'Vol. ToT', 'ex_debt_to_gdp':'Ext. Debt', 'gold_to_gdp':'Curr. res.'},columns={'ape':'5 Yr small'}), 
    ], axis=1).drop(index=['constant'], axis=0)

if save_figures:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", )
        apeLatex = ape.to_latex()
        warnings.filterwarnings("default")
    with open(path_to_output+'ape_latex.tex', 'w') as f:
        f.write(apeLatex)

print(ape)

## Predict spreads for EMBI countries

Method is based on Hull exercise 24.6 in which he calculates the historical one-year hazard rate from cumulative default rates. From there he uses an assumption of recovery rate of 40% to calculate the spread. 

    The spread to compensate for default are 60% of these (red: one-year hazard rate).
    

In [ ]:
l_3yr_full.xLabels

In [ ]:
df_embi[['Date']+l_3yr_full.xLabels].dropna()['Date'].max()

In [ ]:
df_embi.columns.values

In [ ]:
df_logit = df_embi.groupby(['Date','COUNTRY_CODE']).agg({'PCT_TOT_Real_3yr':'first', 'VOL_TOT_Real_3yr':'first','spread':'median',
                                                         'treasury_yield':'first','slope':'first','VIX':'first',
                                                         'rating_rank':'first','RISK_REGION':'first','rating_sp':'first','ex_debt_to_gdp':'first','gold_to_gdp':'first','YTD':'first','constant':'first'}).reset_index()

df_plot = deepcopy(df_logit)

# l_3yr_full.regressio_results
Xembi = df_plot[l_3yr_full.xLabels]

# Predict probabilities of default
yhat = tools.predict(l_3yr_full.logit_res['theta'], Xembi.values)

df_plot['pred_prob_default'] = yhat

# Calculate the predicted spread
p = df_plot['pred_prob_default'].copy()
rr = 0.4 # recovery rate
lgd = 1 - rr  # loss given default
df_plot['pred_spreads'] = p*lgd *10_000

# df_embi.columns.values
df_plot[['pred_prob_default','pred_spreads','spread']]

In [ ]:
df_plot.dropna(subset='pred_spreads')['Date'].value_counts()
df_plot.dropna(subset='pred_spreads')['COUNTRY_CODE'].unique()

In [ ]:
# df_plot['spread_lead'] = df_plot.groupby('COUNTRY_CODE')['spread'].transform(lambda x: x.shift(1))
# df_plot['Date_lead'] = df_plot['Date'] + pd.DateOffset(years=1)

dfPlot = df_plot.loc[(df_plot[['pred_spreads','spread','rating_rank']].notna().all(axis=1)), :] # OBS: Should we use pred_spreads_lead
dfPlot.loc[dfPlot['COUNTRY_CODE']=='ARM',['Date','COUNTRY_CODE','spread','pred_spreads']]

In [ ]:
dfPlot = df_plot.loc[(df_plot[['pred_spreads','spread']].notna().all(axis=1)), :] # OBS: Should we use pred_spreads_lead

# Remove if pred_spreads exceeds 3 std deviations
dfPlot = dfPlot.loc[
    (dfPlot['pred_spreads'] < dfPlot['pred_spreads'].mean() + 3*dfPlot['pred_spreads'].std()) & 
    (dfPlot['pred_spreads'] > dfPlot['pred_spreads'].mean() - 3*dfPlot['pred_spreads'].std()), :]

# store the EMBI spread and predicted spreads in separate variables
spread = dfPlot['spread'].values 
pred_spread = dfPlot['pred_spreads'].values
region = dfPlot['RISK_REGION'].values
year = dfPlot['Date'].dt.year.values
rating_rank = dfPlot['rating_rank'].values
rating = dfPlot['rating_sp'].values
controls = dfPlot[['RISK_REGION','pred_prob_default']].values

# remove where spread exceeds 25000
threshold = 1_000 #2500
mask = (spread > 100) & (spread < threshold) & (region != 'Middle East') & (rating != 'D')
region = region[mask]
pred_spread = pred_spread[mask]
spread = spread[mask]
year = year[mask]
rating_rank = rating_rank[mask]
rating = rating[mask]
controls = controls[mask, :]


In [ ]:
dfPlot = dfPlot.loc[(dfPlot['spread']<1_000)&(dfPlot['spread']>1_00) & (dfPlot['RISK_REGION']!='Middle East') & (dfPlot['rating_sp']!='D'),:]
# dfPlot

In [ ]:
# OLS fit 
x = dfPlot[['pred_spreads','VIX','treasury_yield','slope']].values
add_constant = False
olsReg = sm.OLS(endog=dfPlot['spread'], exog=x).fit(cov_type='HC3') 
olsReg.summary()


In [ ]:
# Create a scatter plot
fig, ax = plt.subplots(figsize=(7, 4))

# Scatter plot of predicted spreads vs. EMBI spread
ax.scatter(pred_spread, spread, color=cmap[0], alpha=0.5, label='Data', s=20)

# OLS fit 
x = np.linspace(pred_spread.min(), pred_spread.max(), 100)
add_constant = False
olsReg = sm.OLS(endog=spread, exog=pred_spread).fit() if not add_constant else sm.OLS(endog=spread, exog=sm.add_constant(pred_spread)).fit()
y_hat = olsReg.predict(x) if not add_constant else olsReg.predict(sm.add_constant(x))
res = olsReg.resid
ax.plot(x, y_hat, color='black', linestyle='--', label='OLS Fit', alpha=0.8)
# Write the coefficient and r-squared value on the plot with italic
ax.text(0.75, 0.25, f'Coefficient: {olsReg.params[0]:.2f}', transform=ax.transAxes, fontsize=10,fontdict={'style':'italic'})
ax.text(0.75, 0.20, f'R-squared: {olsReg.rsquared:.2f}', transform=ax.transAxes, fontsize=10,fontdict={'style':'italic'})

# Set labels
ax.set_xlabel('Predicted Spreads (bps)')
ax.set_ylabel('EMBI Spread (bps)')

# Remove the top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add legend
ax.legend(title='Region', loc='upper right', bbox_to_anchor=(1, 1), fontsize='small', frameon=False,) 

# Set x and y limits
ax.set_ylim(0,threshold*1.05)
ax.set_xlim(pred_spread.min()*(1 - 0.2*sign(pred_spread.min())),pred_spread.max()*(1 + 0.2*sign(pred_spread.max())))
# Gridlines
ax.grid(which='major', linestyle='--', linewidth=0.5)

if save_figures:
    plt.savefig(path_to_output+'scatter_predicted_spreads.png', dpi=300, bbox_inches='tight')


plt.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12, 4))

for i, reg in enumerate(np.unique(region)):
    ax[0].bar(i, np.mean(pred_spread[region==reg]-spread[region==reg]), alpha=1, label=reg, color=cmap[i])
    # ax[0].errorbar(i, np.mean(pred_spread[region==reg]-spread[region==reg]), 
    #                yerr=np.std(pred_spread[region==reg]-spread[region==reg]), fmt='o', color='black')

for i, y in enumerate(np.unique(year)):
    ax[1].bar(i, np.mean(pred_spread[year==y]-spread[year==y]), alpha=1, label=y, color=cmap[i])
    # ax[1].errorbar(i, np.mean(pred_spread[year==y]-spread[year==y]), 
    #                yerr=np.std(pred_spread[year==y]-spread[year==y]), fmt='o', color='black')

for i, rat in enumerate(np.unique(rating_rank)):
    ax[2].bar(i, np.mean(pred_spread[rating_rank==rat]-spread[rating_rank==rat]), alpha=1, label=y, color=cmap[i])
    # ax[2].errorbar(i, np.mean(pred_spread[rating_rank==rat]-spread[rating_rank==rat]), 
    #                yerr=np.std(pred_spread[rating_rank==rat]-spread[rating_rank==rat]), fmt='o', color='black')

# Set xticks 
ax[0].set_xticks(range(len(np.unique(region))))
ax[0].set_xticklabels([reg if ' ' not in reg else reg.replace(' ','\n') for reg in np.unique(region)], rotation=45)
ax[1].set_xticks(range(len(np.unique(year))))
ax[1].set_xticklabels(np.unique(year+1), rotation=45)
ax[2].set_xticks(range(len(np.unique(rating))))
ax[2].set_xticklabels(np.unique(rating)[::-1], rotation=45)


# Set labels
ax[0].set_ylabel('Mean Predicted - Actual Spread')

# Remove spines 
ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[2].spines['top'].set_visible(False)
ax[2].spines['right'].set_visible(False)

# Add horizontal gridlines
ax[0].grid(axis='y', linestyle='--', linewidth=0.5)
ax[1].grid(axis='y', linestyle='--', linewidth=0.5)
ax[2].grid(axis='y', linestyle='--', linewidth=0.5)

if save_figures:
    plt.savefig(path_to_output+'bar_predicted_spreads.png', dpi=300, bbox_inches='tight')

plt.show()

***Below code creates problems as it re-defines spreads...***

In [ ]:
quantiles = np.quantile(controls[:,1], [0, 0.25, 0.5, 0.75, 1], axis=0,)

# Create a bar chart 
fig, ax = plt.subplots(figsize=(7, 4))

for i in range(4):
    quantile = i*0.25
    mask = (controls[:,1] >= quantiles[i]) & (controls[:,1] < quantiles[i+1])
    spread_quantile = spread[mask]
    pred_spread_quantile = pred_spread[mask]
    # Predicted spread for the quantile
    ax.bar(quantile-0.02, np.mean(spread_quantile), width=0.1, label=f'Acutal Spreads', color=cmap[0])
    # ax.errorbar(quantile-0.02, np.mean(spread_quantile), yerr=np.std(spread_quantile), fmt='o', color='black', alpha=0.5)
    # Actual spread for the quantile
    ax.bar(quantile+0.02, np.mean(pred_spread_quantile), width=0.1, label=f'Predicted Spreads', alpha=0.5,color=cmap[1])
    # ax.errorbar(quantile+0.02, np.mean(pred_spread_quantile), yerr=np.std(pred_spread_quantile), fmt='o', color='black', alpha=0.5)
    
# Remvoe spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Set labels
ax.set_xlabel('Quantiles of Predicted Probability of Default')
ax.set_ylabel('Spread (bps)')

# Only unique labels in the legend
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(0.75, 1), fontsize='small', frameon=False)
# Gridlines
ax.grid(which='major', linestyle='--', linewidth=0.5)
# Set x-ticks
ax.set_xticks(np.arange(0, 4/4, 1/4))
# Set x-tick labels
ax.set_xticklabels([f'{i+1}' for i in range(4)])

# Set y-limits
ax.set_ylim(0, max(np.mean(spread_quantile), np.mean(pred_spread_quantile))*1.2)

if save_figures:
    plt.savefig(path_to_output+'bar_predicted_spreads_quantiles.png', dpi=300, bbox_inches='tight')

plt.show()